In [133]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split

np.random.seed(42)

# Création d'un dataset avec des caractéristiques variées
data = {
    'Taille': np.random.uniform(0.1, 2.0, 50),
    'Poids': np.random.uniform(0.5, 100.0, 50),
    'Type_Animal': np.random.choice(['Mammifère', 'Reptile', 'Oiseau', 'Amphibien'], size=50),
    'Regime_Alimentaire': np.random.choice(['Carnivore', 'Omnivore', 'Herbivore'], size=50),
    'Espece_Protegee': np.random.choice(['Oui', 'Non'], size=50),
    'Comportement': np.random.choice(['Pacifique', 'Farouche', 'Territorial', 'Agressif', 'Prédateur'], size=50),
    'Nom_Animal': [f'Animal_{i}' for i in range(50)],
    'Dangerosite': np.random.randint(0, 11, size=50)
}

df = pd.DataFrame(data)

# Définir la nouvelle variable cible en fonction des conditions spécifiées
df['Dangerosite'] = np.where(
    (df['Comportement'] == 'Pacifique'),  # Si le comportement est pacifique
    0,
    np.where(
        ((df['Type_Animal'] == 'Oiseau') &
        (df['Poids'] > 5) &
        ((df['Comportement'] == 'Agressif') | (df['Comportement'] == 'Territorial'))),  # Si les conditions spécifiées pour les oiseaux sont satisfaites
        10,
        np.where(
            ((df['Type_Animal'] == 'Reptile') | (df['Type_Animal'] == 'Amphibien')) &
            ((df['Regime_Alimentaire'] == 'Carnivore') | ((df['Regime_Alimentaire'] == 'Omnivore') &
            ((df['Comportement'] == 'Territorial') | (df['Comportement'] == 'Prédateur') | (df['Comportement'] == 'Agressif')))),  # Si les conditions spécifiées pour reptiles et amphibiens sont satisfaites
            10,
            np.where(
                ((df['Type_Animal'] == 'Mammifère') | (df['Type_Animal'] == 'Reptile')) &
                (df['Taille'] > 0.82) &
                (df['Poids'] > 70) &
                ((df['Regime_Alimentaire'] == 'Carnivore') |
                 ((df['Regime_Alimentaire'] == 'Omnivore') & ((df['Comportement'] == 'Agressif') | (df['Comportement'] == 'Prédateur')))) &
                (df['Espece_Protegee'] == 'Non'),  # Si les conditions spécifiées pour mammifères et reptiles sont satisfaites
                10,
                df['Dangerosite']  # Sinon, conserve la valeur existante
            )
        )
    )
)

# Suppression des colonnes inutiles
df = df.drop('Nom_Animal', axis=1)
# Sauvegarder le DataFrame dans un fichier CSV
df.to_csv('animal_dataset.csv', index=False)

# Encodage one-hot des variables catégoriques
df_encoded = pd.get_dummies(df, columns=['Type_Animal', 'Regime_Alimentaire', 'Espece_Protegee', 'Comportement'], drop_first=True)

# Sauvegarder le DataFrame dans un fichier CSV
df_encoded.to_csv('animal_encoded_dataset.csv', index=False)


# Divisons le dataset en ensembles d'entraînement et de test
X_encoded = df_encoded.drop('Dangerosite', axis=1)
y_encoded = df_encoded['Dangerosite']
X_train_encoded, X_test_encoded, y_train_encoded, y_test_encoded = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)

# Créons un classificateur de base (arbre de décision)
base_classifier = DecisionTreeClassifier(max_depth=3, random_state=42)

# Entraînement avec Bagging
bagging_classifier = BaggingClassifier(base_classifier, n_estimators=5, random_state=42)
bagging_classifier.fit(X_train_encoded, y_train_encoded)
bagging_accuracy = bagging_classifier.score(X_test_encoded, y_test_encoded)
print(f"Accuracy Bagging: {bagging_accuracy}")

# Entraînement avec Pasting
pasting_classifier = BaggingClassifier(base_classifier, n_estimators=5, bootstrap=False, random_state=42)
pasting_classifier.fit(X_train_encoded, y_train_encoded)
pasting_accuracy = pasting_classifier.score(X_test_encoded, y_test_encoded)
print(f"Accuracy Pasting: {pasting_accuracy}")


Accuracy Bagging: 0.6
Accuracy Pasting: 0.5


## Partie Théorique : Ensembles d'Arbres
### Introduction :
Les ensembles d'arbres sont une stratégie puissante en apprentissage automatique, où plusieurs modèles d'arbres sont combinés pour améliorer la performance globale. Deux techniques couramment utilisées dans les ensembles d'arbres sont le Bagging (Bootstrap Aggregating) et le Pasting.

### 1. Bagging (Bootstrap Aggregating) :
![Schéma Bagging](assets\shema_bagging&plasting\Shema_Baggin.jpg)

#### Idée Principale : 
Utilise le Bootstrap Sampling pour créer des ensembles d'entraînement différents pour chaque modèle.

#### Fonctionnement : 
Chaque modèle est formé indépendamment sur son propre ensemble d'entraînement, créé en tirant aléatoirement avec remplacement à partir du jeu de données original.

#### Avantages :
 Réduit la variance, améliore la stabilité, et prévient le surajustement.


##### Étape 1 : Bootstrap Sampling :

Théorie : On crée plusieurs échantillons d'entraînement en tirant aléatoirement avec remplacement à partir du jeu de données original.

Exemple : Imagine que tu as un sac avec des boules numérotées de 1 à 10. Tu pioches plusieurs fois, remets chaque boule tirée dans le sac, et répètes.

##### Étape 2 : Construction des Arbres :

Théorie : Pour chaque échantillon, on construit un arbre de décision, mais à chaque nœud, on ne considère qu'un sous-ensemble aléatoire des caractéristiques.

Exemple : Quand tu poses des questions, tu ne considères pas toujours les mêmes caractéristiques pour chaque arbre. Par exemple, pour un arbre, tu peux demander "a-t-il des ailes?" et pour un autre "a-t-il des pattes?".

##### Étape 3 : Agrégation :

Théorie : Les prédictions de chaque arbre sont combinées par moyenne (régression) ou vote majoritaire (classification) pour obtenir la prédiction finale.

Exemple : Si tu demandes à plusieurs amis de deviner le nombre de bonbons dans un bocal, vous combinez ensuite vos réponses pour obtenir une estimation moyenne.

### 2. Pasting (Bagging sans Bootstrap Sampling) :

#### Idée Principale : 
Similaire au Bagging, mais utilise un échantillonnage sans remplacement pour créer les ensembles d'entraînement.

#### Différence avec le Bagging :
 Chaque échantillon ne peut être dans qu'un seul ensemble d'entraînement.

#### Avantages : 
Réduit la variance, améliore la stabilité, et est utile lorsque le dataset est limité.

### Hyperparamètres Importants :

#### Nombre de Modèles (n_estimators) : 
Le nombre total d'arbres dans l'ensemble.

#### Profondeur d'Arbre (max_depth) : 
La profondeur maximale de chaque arbre.

#### Nombre de Caractéristiques à Considérer (max_features) : 
Le nombre maximum de caractéristiques à considérer pour chaque division.

#### Critère de Division (criterion) :
Indique comment mesurer la qualité d'une division (ex. Gini pour la classification, MSE pour la régression).


Ensemble d'Arbres (Random Forest) :

Fonctionnement Théorique :
Introduction :
Un Random Forest est un ensemble d'arbres de décision. Au lieu d'avoir un seul arbre, on en construit plusieurs, et la prédiction finale est obtenue en moyennant (pour la régression) ou en votant (pour la classification) les prédictions de chaque arbre.

Étape 1 : Bootstrap Sampling :

Théorie : On crée plusieurs échantillons d'entraînement en tirant aléatoirement avec remplacement à partir du jeu de données original.

Exemple : Imagine que tu as un sac avec des boules numérotées de 1 à 10. Tu pioches plusieurs fois, remets chaque boule tirée dans le sac, et répètes.

Étape 2 : Construction des Arbres :

Théorie : Pour chaque échantillon, on construit un arbre de décision, mais à chaque nœud, on ne considère qu'un sous-ensemble aléatoire des caractéristiques.

Exemple : Quand tu poses des questions, tu ne considères pas toujours les mêmes caractéristiques pour chaque arbre. Par exemple, pour un arbre, tu peux demander "a-t-il des ailes?" et pour un autre "a-t-il des pattes?".

Étape 3 : Agrégation :

Théorie : Les prédictions de chaque arbre sont combinées par moyenne (régression) ou vote majoritaire (classification) pour obtenir la prédiction finale.

Exemple : Si tu demandes à plusieurs amis de deviner le nombre de bonbons dans un bocal, vous combinez ensuite vos réponses pour obtenir une estimation moyenne.

Hyperparamètres et Stratégie :
Nombre d'Arbres (n_estimators) :

Importance : Le nombre total d'arbres dans le Random Forest.
Ordre de Grandeur : Typiquement entre 50 et 500 arbres.
Cas Spécifiques : Plus d'arbres améliorent généralement la performance, mais cela nécessite plus de temps de calcul.
Profondeur d'Arbre (max_depth) :

Importance : La profondeur maximale de chaque arbre.
Ordre de Grandeur : Souvent entre 10 et 100.
Cas Spécifiques : Limiter la profondeur peut prévenir le surajustement.
Nombre de Caractéristiques à Considérer (max_features) :

Importance : Le nombre maximum de caractéristiques à considérer pour chaque division.
Ordre de Grandeur : Racine carrée du nombre total de caractéristiques.
Cas Spécifiques : Ajuster cela peut contrôler la diversité des arbres.
Min. d'Échantillons pour Diviser (min_samples_split) :

Importance : Le nombre minimum d'échantillons nécessaires pour effectuer une division.
Ordre de Grandeur : Généralement petit, comme 2 à 5.
Cas Spécifiques : Ajuster cela peut contrôler la complexité des arbres.
Utilisation dans la Régression et la Classification :
Régression : Oui, les Random Forest sont couramment utilisés pour la régression. La prédiction finale est la moyenne des prédictions des arbres.

Classification : Oui, les Random Forest sont également très efficaces pour la classification. La prédiction finale est déterminée par le vote majoritaire des arbres.

Exemple de Code (en Python avec scikit-learn) :

python
Copy code
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score

# Exemple pour la Régression
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
regressor = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
regressor.fit(X_train, y_train)
predictions = regressor.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')

# Exemple pour la Classification
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
classifier = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
classifier.fit(X_train, y_train)
predictions = classifier.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy}')
Ces exemples de code illustrent comment utiliser un Random Forest pour la régression et la classification avec scikit-learn en Python. Vous pouvez ajuster les hyperparamètres selon les besoins spécifiques de votre problème.







In [134]:

from sklearn.neighbors import KNeighborsClassifier

# Bagging (Bootstrap Aggregating) :
## Introduction :
Le Bagging est une technique d'ensemble qui vise à améliorer la stabilité et la précision des modèles en combinant les prédictions de plusieurs modèles indépendants. Il utilise le Bootstrap Sampling pour créer des ensembles d'entraînement différents pour chaque modèle.

### Fonctionnement Théorique :

### Bootstrap Sampling :

Théorie : On crée plusieurs échantillons d'entraînement en tirant aléatoirement avec remplacement à partir du jeu de données original.
Exemple : Si tu as un sac avec des boules numérotées de 1 à 10, tu pioches plusieurs fois, remets chaque boule tirée dans le sac, et répètes.
Entraînement Indépendant :

Théorie : Chaque modèle est entraîné indépendamment sur son ensemble de données d'entraînement spécifique.
Exemple : Imaginons que chaque modèle représente un ami qui essaye de deviner quelque chose. Chacun d'eux a un ensemble d'informations légèrement différent.
Agrégation :

Théorie : Les prédictions de chaque modèle sont combinées par moyenne (pour la régression) ou par vote majoritaire (pour la classification).
Exemple : Si tu demandes à plusieurs amis de deviner le nombre de bonbons dans un bocal, vous combinez ensuite vos réponses pour obtenir une estimation moyenne.
Hyperparamètres et Stratégie :

### Nombre de Modèles (n_estimators) :

Importance : Le nombre total de modèles dans l'ensemble.
Ordre de Grandeur : Typiquement entre 50 et 500 modèles.
Cas Spécifiques : Plus de modèles améliorent généralement la performance, mais cela nécessite plus de temps de calcul.
Profondeur d'Arbre (max_depth) :

Importance : La profondeur maximale de chaque arbre dans les modèles.
Ordre de Grandeur : Souvent entre 10 et 100.
Cas Spécifiques : Limiter la profondeur peut prévenir le surajustement.
Nombre de Caractéristiques à Considérer (max_features) :

Importance : Le nombre maximum de caractéristiques à considérer pour chaque division dans les arbres.
Ordre de Grandeur : Racine carrée du nombre total de caractéristiques.
Cas Spécifiques : Ajuster cela peut contrôler la diversité des modèles.
# Pasting (Bagging sans Bootstrap Sampling) :
## Introduction :
Le Pasting est similaire au Bagging, mais au lieu d'utiliser Bootstrap Sampling, il utilise un échantillonnage sans remplacement pour créer les ensembles d'entraînement pour chaque modèle.

## Différence avec le Bagging :

Bagging : Utilise Bootstrap Sampling (tirage avec remplacement).
Pasting : Utilise un échantillonnage sans remplacement.
### Fonctionnement Théorique :

Le fonctionnement du Pasting est essentiellement le même que le Bagging, mais au lieu de permettre aux mêmes échantillons de se retrouver dans plusieurs ensembles d'entraînement, chaque échantillon ne peut être dans qu'un seul ensemble.
Hyperparamètres et Stratégie :

Les hyperparamètres et la stratégie pour le Pasting sont essentiellement les mêmes que pour le Bagging.
Utilisation dans la Régression et la Classification :

Le Bagging et le Pasting peuvent être utilisés aussi bien pour la régression que pour la classification, en fonction du type de modèle utilisé pour chaque base learner (modèle de base) dans l'ensemble.
Exemple de Code (en Python avec scikit-learn) :

python
Copy code
from sklearn.ensemble import BaggingRegressor, BaggingClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score

# Exemple pour la Régression avec Bagging
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
base_regressor = DecisionTreeRegressor(max_depth=10, random_state=42)
bagging_regressor = BaggingRegressor(base_regressor, n_estimators=100, random_state=42)
bagging_regressor.fit(X_train, y_train)
predictions = bagging_regressor.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error (Bagging): {mse}')

# Exemple pour la Classification avec Pasting
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
base_classifier = DecisionTreeClassifier(max_depth=10, random_state=42)
pasting_classifier = BaggingClassifier(base_classifier, n_estimators=100, bootstrap=False, random_state=42)
pasting_classifier.fit(X_train, y_train)
predictions = pasting_classifier.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy (Pasting): {accuracy}')
Ces exemples de code montrent comment utiliser le Bagging pour la régression et le Pasting pour la classification avec scikit-learn en Python. Vous pouvez ajuster les hyperparamètres selon les besoins spécifiques de votre problème.

In [135]:
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

X, y = make_blobs(n_samples=10000, n_features=10, centers=100,
    random_state=0)

clf = DecisionTreeClassifier(max_depth=None, min_samples_split=2,
    random_state=0)
scores = cross_val_score(clf, X, y, cv=5)
scores.mean()

clf = RandomForestClassifier(n_estimators=10, max_depth=None,
    min_samples_split=2, random_state=0)
scores = cross_val_score(clf, X, y, cv=5)
scores.mean()

clf = ExtraTreesClassifier(n_estimators=10, max_depth=None,
    min_samples_split=2, random_state=0)
scores = cross_val_score(clf, X, y, cv=5)
scores.mean() > 0.999

True

Wisdom of the crowd 
VotingClassifier & VotingRegressor
BagginClassifier & BaginRegressor


In [136]:
model = BaggingClassifier(n_estimators=100)
model.fit(X_train, y_train)

ValueError: could not convert string to float: 'Mammifère'

Accuracy

les limite de l'accuracy : classe imbalance dummyclassifier de sklearn 
matrice de confusion : 
recall = tp / (tp +fn)
precision = tp / (tp +fp)
f1 score = 2*(presicion * recall)/(precision + recall) = 2tp/ (

)
spécificité = tn / fp+tn